In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

## Normalización

In [11]:
def normalizar_fechas(df):
    """
    Elimina las horas de las columnas 'fecha_inicio' y 'fecha_fin', dejando solo 'YYYY-MM-DD'.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'fecha_inicio' y 'fecha_fin'.
        
    Retorna:
        pd.DataFrame: DataFrame con las fechas normalizadas.
    """
    df["fecha_inicio"] = df["fecha_inicio"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    df["fecha_fin"] = df["fecha_fin"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    
    return df


In [12]:
df_api = normalizar_fechas(df_api)
df_api.sample(3)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
20,12698465,IV Festival de carnaval Carabanchel,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28054,Parque de Salvador Allende,11:00,2025-03-01,2025-03-01,Parque de Salvador Allende
40,12693997,Tejiendo Colores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,00:00,2025-03-01,2025-03-31,Centro de Educación Ambiental y Cultural Maris...
44,12686173,Villaverde Bajo. Concurso de Disfraces y Pasac...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,17:30,2025-03-01,2025-03-01,None


## Carga de datos

## Paso 1. Conectar con Postgres

In [13]:
import psycopg2
from psycopg2 import OperationalError

def create_connection():
    try:
        # Configurar la conexión
        conn = psycopg2.connect(
            dbname="project_etl_ibis",
            user="postgres",
            password="admin",
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa")
        return conn
    except OperationalError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Crear la conexión
conn = create_connection()

if conn:
    # Crear un cursor para ejecutar comandos SQL
    cur = conn.cursor()

    # Verificar la conexión
    cur.execute("SELECT version();")
    print(cur.fetchone())

    # Cerrar la conexión
    cur.close()
    conn.close()

Conexión exitosa
('PostgreSQL 17.0 on x86_64-windows, compiled by msvc-19.41.34120, 64-bit',)


## 2. Carga de Datos

### Inserto datos en TABLA CIUDAD

In [14]:
import psycopg2

# Conectar a la base de datos
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Obtener el nombre de la ciudad desde el DataFrame (suponiendo que todas son "Madrid")
nombre_ciudad = "Madrid"

# 1️⃣ Verificar si la ciudad ya existe
cur.execute("SELECT id_ciudad FROM ciudad WHERE nombre_ciudad = %s", (nombre_ciudad,))
result = cur.fetchone()

if result:
    id_ciudad = result[0]  # Si ya existe, obtenemos su ID
    print(f"La ciudad '{nombre_ciudad}' ya existe con id_ciudad = {id_ciudad}")
else:
    # 2️⃣ Si no existe, insertamos la ciudad y obtenemos el id_ciudad
    cur.execute("INSERT INTO ciudad (nombre_ciudad) VALUES (%s) RETURNING id_ciudad;", (nombre_ciudad,))
    id_ciudad = cur.fetchone()[0]
    conn.commit()
    print(f"Se insertó la ciudad '{nombre_ciudad}' con id_ciudad = {id_ciudad}")

# Cerrar conexión
cur.close()
conn.close()


La ciudad 'Madrid' ya existe con id_ciudad = 1


QUERY

In [15]:
# Conectarse a PostgreSQL
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Obtener todas las ventas
cur.execute("SELECT * FROM ciudad;")

# Mostrar los resultados
for row in cur.fetchall():
    print(row)

# Cerrar conexión
cur.close()
conn.close()

(1, 'Madrid')


### Cargar datos en TABLA HOTELES

In [16]:
df_completo = pd.read_csv(r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\dfcompleto.csv", parse_dates=["fecha_reserva", "inicio_estancia", "final_estancia"])

In [17]:
df_completo.rename(columns={"valoración": "valoracion"}, inplace=True)  # Renombrar la columna
df_completo.dtypes

id_reserva                 object
id_cliente                  int64
nombre                     object
apellido                   object
mail                       object
competencia                  bool
fecha_reserva      datetime64[ns]
inicio_estancia    datetime64[ns]
final_estancia     datetime64[ns]
id_hotel                    int64
precio_noche              float64
nombre_hotel               object
valoracion                float64
ciudad                     object
dtype: object

In [18]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# 1️⃣ Obtener la ciudad del DataFrame (asumiendo que todas son "Madrid")
nombre_ciudad = df_completo["ciudad"].unique()[0]  # Si hay más ciudades, necesitarás un bucle

# 2️⃣ Verificar si la ciudad ya existe
cur.execute("SELECT id_ciudad FROM ciudad WHERE nombre_ciudad = %s", (nombre_ciudad,))
result = cur.fetchone()

if result:
    id_ciudad = result[0]  # Recuperar el ID de la ciudad
    print(f"La ciudad '{nombre_ciudad}' ya existe con id_ciudad = {id_ciudad}")
else:
    # Insertar la ciudad si no existe
    cur.execute("INSERT INTO ciudad (nombre_ciudad) VALUES (%s) RETURNING id_ciudad;", (nombre_ciudad,))
    id_ciudad = cur.fetchone()[0]
    conn.commit()
    print(f"Se insertó la ciudad '{nombre_ciudad}' con id_ciudad = {id_ciudad}")

# 3️⃣ Preparar los datos de hoteles
df_hoteles = df_completo[["nombre_hotel", "competencia", "valoracion"]].copy()
df_hoteles["id_ciudad"] = id_ciudad  # Asignar el id_ciudad obtenido
df_hoteles["competencia"] = df_hoteles["competencia"].astype(bool)

# 4️⃣ Eliminar duplicados para evitar insertar el mismo hotel varias veces
df_hoteles = df_hoteles.drop_duplicates(subset=["nombre_hotel"])

# 5️⃣ Convertir DataFrame a lista de tuplas para la inserción
records = df_hoteles[["nombre_hotel", "competencia", "valoracion", "id_ciudad"]].to_records(index=False)
records_list = [(str(nombre_hotel), bool(competencia), float(valoracion), int(id_ciudad)) for nombre_hotel, competencia, valoracion, id_ciudad in records]

# 6️⃣ Insertar los datos en la tabla `hoteles`
query = """
    INSERT INTO hoteles (nombre_hotel, competencia, valoracion, id_ciudad) 
    VALUES (%s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Hoteles insertados correctamente.")

La ciudad 'Madrid' ya existe con id_ciudad = 1
Hoteles insertados correctamente.


## Tabla clientes 

In [19]:
df_completo.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'valoracion', 'ciudad'],
      dtype='object')

In [20]:
df_cliente = df_completo[["id_cliente", "nombre", "apellido", "mail"]].copy()

In [21]:
# Convertir id_cliente a string antes de insertarlo
df_cliente["id_cliente"] = df_cliente["id_cliente"].astype(str)


In [22]:
df_cliente.dtypes

id_cliente    object
nombre        object
apellido      object
mail          object
dtype: object

In [23]:
df_cliente = df_cliente.drop_duplicates(subset=["id_cliente"])

In [24]:
# Reabrir la conexión
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas
records = df_cliente[["id_cliente", "nombre", "apellido", "mail"]].to_records(index=False)
records_list = list(records)

# Insertar múltiples registros con ejecuta_many()
query = """
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")

UniqueViolation: llave duplicada viola restricción de unicidad «clientes_pkey»
DETAIL:  Ya existe la llave (id_cliente)=(2).


In [25]:
conn.rollback()

rollback

In [ ]:
df_cliente["id_cliente"].count()

14905

QUERY

SELECT r.id_reserva, c.id_cliente
FROM reservas r
LEFT JOIN clientes c ON r.id_cliente = c.id_cliente
WHERE c.id_cliente IS NULL;  -- Detecta reservas sin cliente


TABLA RESERVAS

In [26]:
df_reservas = df_completo[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel"]].copy()

In [27]:
df_reservas["fecha_reserva"] = pd.to_datetime(df_reservas["fecha_reserva"], errors='coerce')
df_reservas["inicio_estancia"] = pd.to_datetime(df_reservas["inicio_estancia"], errors='coerce')
df_reservas["final_estancia"] = pd.to_datetime(df_reservas["final_estancia"], errors='coerce')


In [28]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()


el problema q me daba es q la fecha reserva tenia q crear cuando extraje los datos una columna poniendo la fecha del estract

In [29]:
from datetime import datetime, timedelta
fecha_ayer = pd.to_datetime(datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

# Reemplazar los valores NaT en la columna 'fecha_reserva' con la fecha de ayer
df_reservas['fecha_reserva'] = df_reservas['fecha_reserva'].fillna(fecha_ayer)

completar datos inicio estancia y fin estancia 
precio noche


In [30]:
# Definir la fecha con la que se rellenarán los valores nulos
fecha_reemplazo = '2025-03-01'

# Rellenar los valores nulos en la columna 'inicio_estancia' con la fecha '2025-03-01'
df_reservas['inicio_estancia'] = df_reservas['inicio_estancia'].fillna(fecha_reemplazo)


In [31]:
# Calcular la moda de la columna 'final_estancia'
moda_final_estancia = df_reservas['final_estancia'].mode()[0]
print(moda_final_estancia)

2025-03-02 00:00:00


In [32]:
# Rellenar los valores nulos en la columna 'final_estancia' con la moda
df_reservas['final_estancia'] = df_reservas['final_estancia'].fillna(moda_final_estancia)

In [33]:
df_reservas['final_estancia'].value_counts()

final_estancia
2025-03-02    15000
Name: count, dtype: int64

In [34]:
# Calcular la mediana de la columna 'precio_noche'
mediana_precio_noche = df_reservas['precio_noche'].median()

# Rellenar los valores nulos en la columna 'precio_noche' con la mediana
df_reservas['precio_noche'] = df_reservas['precio_noche'].fillna(mediana_precio_noche)

In [ ]:
records_list = df_reservas.values.tolist()
records_list ## crear una lista d lsitas 

In [36]:
df_reservas['id_cliente'] = df_reservas['id_cliente'].astype(str)

usar for iter rows porq el records lo convierte en string y luego tienes q volver a transformar

In [37]:

#✅ Convertir fechas a string en formato "YYYY-MM-DD"
#df_reservas["fecha_reserva"] = df_reservas["fecha_reserva"].dt.strftime('%Y-%m-%d')
#df_reservas["inicio_estancia"] = df_reservas["inicio_estancia"].dt.strftime('%Y-%m-%d') ## por que si lo hice antes tengo q volver a pasarlo a datatime
#df_reservas["final_estancia"] = df_reservas["final_estancia"].dt.strftime('%Y-%m-%d')

# ✅ Asegurar que precio_noche es un float positivo

# Convertir DataFrame a lista de tuplas para la inserción


# Insertar en la tabla `reservas`
query = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)"""

cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Reservas insertadas correctamente.")


UniqueViolation: llave duplicada viola restricción de unicidad «reservas_pkey»
DETAIL:  Ya existe la llave (id_reserva)=(40c4cb55-d1f5-407b-832f-4756b8ff77b4).


In [38]:
conn.rollback()

TABLA EVENTOS

In [48]:
df_api.dtypes

id_evento        object
nombre_evento    object
url_evento       object
codigo_postal    object
direccion        object
horario          object
fecha_inicio     object
fecha_fin        object
organizacion     object
dtype: object

In [58]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_evento      45 non-null     object        
 1   nombre_evento  45 non-null     object        
 2   url_evento     45 non-null     object        
 3   codigo_postal  45 non-null     object        
 4   direccion      45 non-null     object        
 5   horario        45 non-null     object        
 6   fecha_inicio   45 non-null     datetime64[ns]
 7   fecha_fin      45 non-null     datetime64[ns]
 8   organizacion   45 non-null     object        
dtypes: datetime64[ns](2), object(7)
memory usage: 3.3+ KB


In [39]:
df_api['codigo_postal'] = df_api['codigo_postal'].fillna(0)
df_api['organizacion'] = df_api['organizacion'].fillna("no se encuentra")

In [40]:
df_api['codigo_postal'].value_counts()

codigo_postal
0        15
28045     5
28022     4
28009     3
28021     3
28015     2
28026     2
28041     2
28028     1
28037     1
28011     1
28035     1
28042     1
28054     1
28017     1
28001     1
28020     1
Name: count, dtype: int64

In [41]:
df_api["fecha_inicio"] = pd.to_datetime(df_api['fecha_inicio'], errors='coerce')
df_api["fecha_fin"] = pd.to_datetime(df_api['fecha_fin'], errors='coerce')

In [42]:
df_api['id_ciudad'] = 1
df_api.dtypes

id_evento                object
nombre_evento            object
url_evento               object
codigo_postal            object
direccion                object
horario                  object
fecha_inicio     datetime64[ns]
fecha_fin        datetime64[ns]
organizacion             object
id_ciudad                 int64
dtype: object

ABRO CONEXION

In [43]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

records_list = df_api.values.tolist()


# Insertar en la tabla `reservas`
query = """
    INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Reservas insertadas correctamente.")



Reservas insertadas correctamente.


2. query

In [46]:
# Conectarse a PostgreSQL
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Ejecutar la consulta para contar el número de hoteles
cur.execute("SELECT COUNT(*) FROM hoteles;")

# Obtener el resultado
total_hoteles = cur.fetchone()[0]
print(f"Total de hoteles en la base de datos: {total_hoteles}")

# Cerrar la conexión
cur.close()
conn.close()


Total de hoteles en la base de datos: 261
